In [6]:
import os
import snowflake.connector as sf
import pandas as pd
import access
import sqlite3


# O caminho do diretório onde você deseja criar os arquivos SQL
output_directory = "C:/ProjetosPython/EDD-DL/Gerar_Script_Sql/arquivos/procs_carga_full_curated/"

# Certifique-se de que o diretório exista, senão o crie
os.makedirs(output_directory, exist_ok=True)

# Conexão SQLite        C:/ProjetosPython/EDD-DL/Gerar_scripts/database/config_scripts.db
conn = sqlite3.connect('C:/ProjetosPython/EDD-DL/Gerar_Script_Sql/database/config_scripts.db')
cursor = conn.cursor()

# Leitura das tabelas ativas do SQLite
query_tabelas_full = "SELECT TABELA FROM config_tabelas_full WHERE ATIVO = 'true';"
df_tabelas_full = pd.read_sql_query(query_tabelas_full, conn)
#print(df_tabelas_full)


for tabelas_full in df_tabelas_full['TABELA']:

    query_estrutura_sf_t= f"""
        SELECT COLUMN_NAME
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_CATALOG = 'RESSARCIMENTO'
        AND TABLE_SCHEMA = 'TRANSIENT'     
        AND COLUMN_NAME NOT IN ('ID_ORIGINAL', 'ID_DATABASE')
        AND TABLE_NAME = 'STG_{tabelas_full.upper()}'
        ORDER BY ORDINAL_POSITION;
    """

    query_estrutura_sf_c= f"""
        SELECT COLUMN_NAME
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_CATALOG = 'RESSARCIMENTO'
        AND TABLE_SCHEMA = 'CURATED_DATA'        
        AND COLUMN_NAME NOT IN ('ID','ID_DATABASE')
        AND TABLE_NAME = '{tabelas_full.upper()}'
        ORDER BY ORDINAL_POSITION;
    """
    #print(query_estrutura_sf_c)
    #print(query_estrutura_sf_t)
    try:
        # Obtém a estrutura das colunas CURATED e STAGE no Snowflake
        with sf.connect(
            user=access.USER_SF,
            password=access.PWD_SF,
            account=access.ACCOUNT_SF
        ) as sfConnection:
            sfq = sfConnection.cursor()
            # Adicione o comando USE DATABASE aqui
            sfq.execute("USE DATABASE RESSARCIMENTO")
           
             # Obtém a estrutura das colunas CURATED no Snowflake  
            sfq.execute(query_estrutura_sf_c)
            colunas_sf_c = sfq.fetchall()
            #print(colunas_sf_c)

            # Obtém a estrutura das colunas STAGE no Snowflake  
            sfq.execute(query_estrutura_sf_t)
            colunas_sf_t = sfq.fetchall()
            #print(colunas_sf_t)

    except Exception as err:
        print(err)      

    # Transforme as colunas obtidas em uma lista de nomes
    colunas_curated = ', '.join([col[0] for col in colunas_sf_c])
    colunas_stage = ', '.join([f"STG_{tabelas_full}.{col[0]}" for col in colunas_sf_t])
    
    # Criação do conteúdo do arquivo SQL
    sql_content = f"""CREATE OR REPLACE PROCEDURE RESSARCIMENTO.TRANSIENT.P_STG_{tabelas_full}_FULL()
RETURNS VARCHAR(16777216)
LANGUAGE SQL
EXECUTE AS OWNER
AS '

BEGIN 

--APAGA TUDO DA CURATED
    execute immediate ''
    DELETE
    FROM RESSARCIMENTO.CURATED_DATA.{tabelas_full}
    WHERE ID_DATABASE = (
        SELECT ID_DATABASE 
        FROM TRANSIENT.STG_{tabelas_full}
        LIMIT 1
    ); '';

--INSERE NA CURATED                  
    execute immediate ''
    INSERT INTO RESSARCIMENTO.CURATED_DATA.{tabelas_full} ({colunas_curated}, ID_DATABASE)
    SELECT {colunas_stage}, ID_DATABASE FROM RESSARCIMENTO.TRANSIENT.STG_{tabelas_full};'';
                        
END  '; 
    """
    #print(sql_content)
    output_file_name = tabelas_full + "_full.sql"

    # Caminho completo do arquivo
    output_path = os.path.join(output_directory, output_file_name)
    with open(output_path, "a", encoding='utf8') as file:
         file.write(sql_content)
         file.close()
    print("Arquivos SQL gerados com sucesso! " +str(output_file_name))


Arquivos SQL gerados com sucesso! motivos_fat_full.sql
Arquivos SQL gerados com sucesso! pbm_vendas_pedidos_full.sql
Arquivos SQL gerados com sucesso! pbm_vendas_repostas_full.sql
Arquivos SQL gerados com sucesso! produtos_pedidos_full.sql
Arquivos SQL gerados com sucesso! produtos_notas_full.sql
Arquivos SQL gerados com sucesso! clientes_area_full.sql
Arquivos SQL gerados com sucesso! sortimentos_full.sql
Arquivos SQL gerados com sucesso! tipos_cliente_full.sql
Arquivos SQL gerados com sucesso! sub_produtos_pedidos_full.sql
Arquivos SQL gerados com sucesso! divisao_produto_full.sql
Arquivos SQL gerados com sucesso! condicoes_fat_full.sql
Arquivos SQL gerados com sucesso! grupos_clientes_full.sql
Arquivos SQL gerados com sucesso! area_full.sql
Arquivos SQL gerados com sucesso! familia_full.sql
Arquivos SQL gerados com sucesso! notas_sub_pedidos_full.sql
Arquivos SQL gerados com sucesso! sub_pedidos_full.sql
Arquivos SQL gerados com sucesso! pedido_direto_produtos_full.sql
Arquivos SQL 